In [ ]:


#data_df = pd.read_csv('/content/banknote_net.csv')
#data_feather = pd.read_feather('/content/banknote_net.feather')



#TRAIN FROM EMBEDDING

In [ ]:
import argparse

#a list of allowable currency choice
currency_choices = [
    'AUD',
    'BRL',
    'CAD',
    'EUR',
    'GBP',
    'INR',
    'JPY',
    'MXN',
    'PKR',
    'SGD',
    'TRY',
    'USD',
    'NZD',
    'NNR',
    'MYR',
    'IDR',
    'PHP'
]

def parse_arguments():
  #parse argument for shallow classifier training.
  #return argparse parsed arguments.

  #parser arguments and load data
  parser = argparse.ArgumentParser(description="Train model from embeddings.")
  parser.add_argument(
      "--currency",
      "--c",
      type=str,
      choices=currency_choices,
      help = "String of currency for which to train shallow classifier",
      required=True,
  )
  parser.add_argument(
      "--bsize",
      "--b",
      type=int,
      help="Batch size for shallow classifier",
      default=128,
  )
  parser.add_argument(
      "--epochs",
      "--e",
      type=int,
      help="Number of epochs for training shallow top classifier",
      default = 25,
  )
  parser.add_argument(
      "--dpath",
      "--d",
      type=str,
      help="Path to .feather Banknote net embeddings",
      default="/content/banknote_net.feather",
  )
  
  return parser.parse_args()



#training model


In [ ]:
PATH = "/content/banknote_net.feather"
data = pd.read_feather(PATH)
len(data.Currency.unique())



17

In [ ]:
CURRENCY = [
    'AUD',
    'BRL',
    'CAD',
    'EUR',
    'GBP',
    'INR',
    'JPY',
    'MXN',
    'PKR',
    'SGD',
    'TRY',
    'USD',
    'NZD',
    'NNR',
    'MYR',
    'IDR',
    'PHP',
]
data.Currency

0        AUD
1        AUD
2        AUD
3        AUD
4        AUD
        ... 
24821    USD
24822    USD
24823    USD
24824    USD
24825    USD
Name: Currency, Length: 24826, dtype: object

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
def main():
  "trains shallow classifier using embeddings"

 # args = parse_arguments()
  CURRENCY = [
    'AUD',
    'BRL',
    'CAD',
    'EUR',
    'GBP',
    'INR',
    'JPY',
    'MXN',
    'PKR',
    'SGD',
    'TRY',
    'USD',
    'NZD',
    'NNR',
    'MYR',
    'IDR',
    'PHP',
]
  BATCH_SIZE = 128
  NB_EPOCH = 25
  PATH = "/content/banknote_net.feather"

  #load data from embedding
  data = pd.read_feather(PATH)
  data.Currency.unique()
  #data = data[data.Currency == CURRENCY]
  
  data= data.sample(frac = 1)
  labels = data.pop(
      "Denomination"
  )
  print("load data from embedding:ok")
  labels = labels.astype("category")
  labels_encoded = pd.get_dummies(labels)
  data = data.iloc[:,:-1] #keep the embedding
  print("data for embedding: ok")
  #define dataset and shallow model
  NUM_CLASSES  = len(labels.unique())
  NB_TRAINING_SAMPLES = len(data)

  input = Input(shape=(256,))
  x = Dense(128,activation='relu')(input)
  x = Dropout(0.5)(x)
  x = Dense(NUM_CLASSES, activation='softmax')(x)
  model = Model(inputs=input,outputs=x)
  model.summary()
  print("model build: ok")
  checkpoint = ModelCheckpoint(
      filepath = ("/content/shallow_classifier.h5"),
      monitor = 'val_acc',
      save_best_only=True,
  )

  #compile and fit
  model.compile(loss='categorical_crossentropy',
                optimizer = tf.keras.optimizers.Adam(lr=1e-3),
                metrics=['acc',
                         tf.keras.metrics.Precision(),
                         tf.keras.metrics.Recall(),
                         ],
                )
  model.fit(x=data.values,
            y = labels_encoded.values,
            steps_per_epoch = NB_TRAINING_SAMPLES//BATCH_SIZE,
            epochs = NB_EPOCH,
            validation_split = 0.2,
            callbacks = [checkpoint],
            )


In [ ]:
main()

load data from embedding:ok
data for embedding: ok
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256)]             0         
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 56)                7224      
                                                                 
Total params: 40,120
Trainable params: 40,120
Non-trainable params: 0
_________________________________________________________________
model build: ok


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/25
193/193 [==============================] - 3s 8ms/step - loss: 2.4899 - acc: 0.3665 - precision: 0.8278 - recall: 0.1697 - val_loss: 1.1287 - val_acc: 0.7437 - val_precision: 0.9658 - val_recall: 0.5226
Epoch 2/25
193/193 [==============================] - 1s 6ms/step - loss: 1.2195 - acc: 0.6708 - precision: 0.8986 - recall: 0.5057 - val_loss: 0.6948 - val_acc: 0.8313 - val_precision: 0.9700 - val_recall: 0.7217
Epoch 3/25
193/193 [==============================] - 1s 6ms/step - loss: 0.9176 - acc: 0.7514 - precision: 0.9133 - recall: 0.6342 - val_loss: 0.5547 - val_acc: 0.8582 - val_precision: 0.9661 - val_recall: 0.7749
Epoch 4/25
193/193 [==============================] - 1s 6ms/step - loss: 0.7784 - acc: 0.7833 - precision: 0.9156 - recall: 0.6918 - val_loss: 0.4861 - val_acc: 0.8681 - val_precision: 0.9652 - val_recall: 0.8033
Epoch 5/25
193/193 [==============================] - 1s 6ms/step - loss: 0.6882 - acc: 0.8032 - precision: 0.9229 - recall: 0.7233 - val_loss: 